In [1]:
import numpy as np
from scipy.optimize import curve_fit, leastsq

In [65]:
class ndfit(object):
    def __init__(self, ivals, dvals, dm):
        self.ivals = ivals
        self.dvals = dvals
        self.dm = dm
        self.npars = self.dm+1
        self.npts = len(self.dvals)
    
    def fplane(self, x, *cpars):
        cpars = np.array(cpars)
        xp = np.zeros(self.npars)
        xp[1:self.npars] = x[0:self.dm]
        xp[0] = 1.0
        return np.sum(xp * cpars)

    def objfun(self, pars):
        fvals = np.array([dv - self.fplane(iv,pars) for dv, iv in zip(self.dvals, self.ivals)])
        return fvals

    def dolsq(self):
        xini = np.zeros(self.npars)
        xini[0] = np.average(self.dvals)
        # Find independent values at average of dependent var
        iave = np.abs(self.dvals-xini[0]).argmin()
        iv_ave = self.ivals[iave]
        ivt = np.transpose(self.ivals)
        for i in np.arange(1, self.npars):
            ivti = ivt[i-1]
            # Find maximum value along independent axis
            iv_max_i = ivti.argmax()
            iv_max = ivti[iv_max_i]
            # Find corresponding dependent variable value
            dv_max = self.dvals[iv_max_i]
            # Find minimum value along independent axis
            iv_min_i = ivti.argmin()
            iv_min = ivti[iv_min_i]
            # Find corresponding dependent variable value
            dv_min = self.dvals[iv_min_i]
            # Estimate slope
            xini[i] = (dv_max - dv_min)/(iv_max - iv_min)/self.dm
        print(xini)
        popt, pcov, idict, mesg, ierr = leastsq(self.objfun, xini, full_output=True, xtol=1.e-20, ftol=1.e-16)
        print(popt)
        print(pcov)
        for k in idict.keys():
            print('{}: {}'.format(k, idict[k]))
        print(mesg)
        print(ierr)

In [80]:
# 3-D
xvec = np.linspace(0, 10)
yvec = np.linspace(0, 10)
zvec = np.linspace(0, 10)
print(xvec)
xx, yy, zz = np.meshgrid(xvec, yvec, zvec)
xflat = xx.flatten()
yflat = yy.flatten()
zflat = zz.flatten()
ivars = np.transpose(np.array([xflat, yflat, zflat]))
dvars = 5 + 2*xx + 3*yy + 4*zz
dvflat = dvars.flatten()

[  0.           0.20408163   0.40816327   0.6122449    0.81632653
   1.02040816   1.2244898    1.42857143   1.63265306   1.83673469
   2.04081633   2.24489796   2.44897959   2.65306122   2.85714286
   3.06122449   3.26530612   3.46938776   3.67346939   3.87755102
   4.08163265   4.28571429   4.48979592   4.69387755   4.89795918
   5.10204082   5.30612245   5.51020408   5.71428571   5.91836735
   6.12244898   6.32653061   6.53061224   6.73469388   6.93877551
   7.14285714   7.34693878   7.55102041   7.75510204   7.95918367
   8.16326531   8.36734694   8.57142857   8.7755102    8.97959184
   9.18367347   9.3877551    9.59183673   9.79591837  10.        ]


In [81]:
fitter = ndfit(ivars, dvflat, 3)

In [82]:
fitter.dolsq()

[ 50.           0.66666667   1.           1.33333333]
[ 5.  2.  3.  4.]
[[  7.71764707e-05  -4.61176471e-06  -4.61176470e-06  -4.61176472e-06]
 [ -4.61176471e-06   9.22352942e-07   1.18711211e-16  -2.13900383e-16]
 [ -4.61176470e-06   1.18711112e-16   9.22352941e-07  -3.33116353e-16]
 [ -4.61176472e-06  -2.13900065e-16  -3.33116274e-16   9.22352943e-07]]
qtf: [ 0.  0.  0.  0.]
nfev: 15
fjac: [[ -2.05162952e+03   0.00000000e+00   0.00000000e+00 ...,  -4.87417437e-03
   -4.87417437e-03  -4.87417437e-03]
 [ -1.52317949e+03  -1.37444822e+03   0.00000000e+00 ...,  -1.87403027e-03
   -1.87403027e-03  -1.87403027e-03]
 [ -1.52317949e+03  -5.85634460e+02   1.24343886e+03 ...,   8.60902813e-04
    1.02502953e-03   1.18915644e-03]
 [ -3.04635898e+02  -1.17126892e+02   7.43030538e+01 ...,  -6.69775579e-03
   -6.80629629e-03  -6.91483691e-03]]
fvec: [ 0.  0.  0. ...,  0.  0.  0.]
ipvt: [3 2 4 1]
The cosine of the angle between func(x) and any column of the
  Jacobian is at most 0.000000 in absolut